In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

# Quick PP Interpretation

In [13]:
from quick_pp.lithology.carbonate import Carbonate
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.saturation import *
from quick_pp.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_typing, estimate_vsh_gr
from quick_pp.plotter import plotly_log
from quick_pp.utils import zone_flagging

In [ ]:
carbonate_type =  'limestone'  # 'dolostone'  #
model = 'single'  # 'double'  #
method = 'neu_den'  # 'pef_den'  #

df = project.get_all_data()
for well_name, well_data in df.groupby('WELL_NAME'):
    # Load well object
    well = project.get_well(well_name)

    # Clean up data
    well_data = badhole_flagging(well_data)
    well_data = mask_outside_threshold(well_data, fill=True)

    # Initialize lithology model
    args = {
        'litho_model': 'carb',
        'dry_sand_point': (.0, 2.71) if carbonate_type == 'limestone' else (.0, 2.81),  # Dry calcite point
        'dry_clay_point': (.3, 2.7),
        'silt_line_angle': 116,
        'wet_clay_point': (0.43, 2.6),
        'sw_water_salinity': 5000,
        'sw_m': 1.8,
        'sw_n': 2,
        'hc_corr_angle': 50,
        'hc_buffer': 0.1,
        'ressum_cutoffs': dict(
            VSHALE=.5,
            PHIT=0,
            SWT=1
        ),
    }
    carb_model = Carbonate(**args)
    vclw, vcalc, vdolo = carb_model.estimate_lithology(
        well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], pef=well_data['PEF'],
        model=model, method=method, normalize= True, carbonate_type=carbonate_type
    )
    args.update(carb_model.__dict__)
    well.update_config(args)  # Save lithology model to well

    # Implement hydrocarbon correction
    nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
        well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'],
        dry_sand_point=args['dry_sand_point'],
        dry_clay_point=args['dry_clay_point'],
        corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
    )

    # Estimate lithology
    vsh_gr = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=110)
    carb_model = Carbonate(**args)
    vclw, vcalc, vdolo = carb_model.estimate_lithology(
        nphihc, rhobhc,  gr=well_data['GR'], pef=well_data['PEF'], vsh_gr=vsh_gr,
        model=model, method=method, normalize= True, carbonate_type=carbonate_type
    )

    # PHID needs unnormalized lithology
    vclw_un, vcalc_un, vdolo_un = Carbonate(**args).estimate_lithology(
        nphihc, rhobhc, gr=well_data['GR'], pef=well_data['PEF'],
        model=model, method=method, normalize= False, carbonate_type=carbonate_type
    )
    rho_ma = rho_matrix(vclay=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un)
    phid = phit = density_porosity(rhobhc, rho_ma)

    # Estimate water saturation
    temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'metric')  # Arbitrarily use MD instead of TVD
    water_salinity = args['sw_water_salinity']
    rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
    b = estimate_b_waxman_smits(temp_grad, rw)
    qv = estimate_qv(vclw, phit, cec_clay=.1)

    m = args['sw_m']

    swt = waxman_smits_saturation(well_data['RT'], rw, phit, qv)
    swt_a = archie_saturation(well_data['RT'], rw, phit, 1, 2, 2)

    # Estimate permeability
    perm = choo_permeability(vclw, 0, phit)
    # perm_kc = kozeny_carman_permeability(phit, Swirr)
    # perm_t = timur_permeability(phit, Swirr)
    # perm_c = coates_permeability(phit, Swirr)
    # perm_tx = tixier_permeability(phit, Swirr)

    # Estimate rock types
    rock_flag = rock_typing(vclw, higher_is_better=False)

    # Update ZONES
    well_data = zone_flagging(well_data)
    zones = well_data.ZONES

    # Estimate reservoir summary
    ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm, zones, cutoffs=args['ressum_cutoffs'])

    # Update data in the project
    well_data['NPHI_HC'] = nphihc
    well_data['RHOB_HC'] = rhobhc
    well_data['HC_FLAG'] = hc_flag
    well_data['VCALC'] = vcalc
    well_data['VDOLO'] = vdolo
    well_data['VCLW'] = vclw
    well_data['PHIT'] = phit
    well_data['PHID'] = phid
    well_data['RW'] = rw
    well_data['B'] = b
    well_data['Qv'] = qv
    well_data['M'] = m
    well_data['SWT'] = swt
    well_data['PERM'] = perm
    well_data['ROCK_FLAG'] = rock_flag
    well_data['VHC'] = phit * (1 - swt)

    # Save the results to the well
    well.update_data(well_data)
    well.update_ressum(ressum_df)
    project.save_well(well)


In [ ]:
# Plot individual results
well_name = 'M3'
well = project.get_well(well_name)

fig = plotly_log(well.data, well.depth_uom)
fig.show()

In [18]:
# project.save()